# How to use Deep Learning to predict Tomorrow's Temperature with Feedforward Network
## Multivariate Multiple Linear Perceptron
### Author : Mohamed JELIDI, Email : jelidi.mohamad@gmail.com

When we say **Multivariate** time series data means data where there is **more than one observation** for each time step.

A problem may have two or more parallel input time series and an output time series that is dependent on the input time series. The input time series are parallel because each series has an observation at the same time step. We can demonstrate this with a simple example of two parallel input time series where the output series is the simple addition of the input series.

In [5]:
from numpy import array
# define input sequence
in_seq1 = array([100, 200, 300, 400, 500, 600, 700, 800, 900])
in_seq2 = array([150, 250, 350, 450, 550, 650, 750, 850, 950])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
print(out_seq)

[ 250  450  650  850 1050 1250 1450 1650 1850]


## Preprocessing data

We can reshape these three arrays into onesingle dataset where each row represents one time step and each column is a separate time series. Infact, This is a standard way of storing parallel time series in a Comma Separated Value file.

In [8]:
from numpy import hstack
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
print(dataset)

[[ 100  150  250]
 [ 200  250  450]
 [ 300  350  650]
 [ 400  450  850]
 [ 500  550 1050]
 [ 600  650 1250]
 [ 700  750 1450]
 [ 800  850 1650]
 [ 900  950 1850]]


Now, we can see our dataset with one row per time step and one column for each of the two input and one output parallel time series.

Like the univariate time series, we must structure these data into samples in the form of inputs and outputs. We need to split the data into samples maintaining the order of observations across the two input sequences.

In [10]:
# multivariate data preparation
from numpy import array
from numpy import hstack

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
in_seq1 = array([100, 200, 300, 400, 500, 600, 700, 800, 900])
in_seq2 = array([150, 250, 350, 450, 550, 650, 750, 850, 950])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

(7, 3, 2) (7,)
[[100 150]
 [200 250]
 [300 350]] 650
[[200 250]
 [300 350]
 [400 450]] 850
[[300 350]
 [400 450]
 [500 550]] 1050
[[400 450]
 [500 550]
 [600 650]] 1250
[[500 550]
 [600 650]
 [700 750]] 1450
[[600 650]
 [700 750]
 [800 850]] 1650
[[700 750]
 [800 850]
 [900 950]] 1850


Running the example first prints the shape of the X and y components. We can see that the X component has a three-dimensional structure. 

1.   The first dimension is the number of samples, in this case 7.
2.   The second dimension is the number of time steps per sample, in this case 3 (the value specified to the function)
3.   The last dimension specifies the number of parallel time series or the number of variables, in this case 2 for the two parallel series.

# MLP Model

Before we can fit an MLP on this data, we should flatten the shape of the input samples. **MLPs require that the shape of the input portion of each sample is a vector**. We can flatten the temporal structure of each input sample, so that:

[[100 150]  [200 250]  [300 350]] becomes [100, 150, 200, 250, 300, 350]

First, we calculate the length of each input vector as the number of time steps multiplied by the number of features or time series. We can then use this vector size to reshape the input.

In [12]:
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))
print(X)

[[100 150 200 250 300 350]
 [200 250 300 350 400 450]
 [300 350 400 450 500 550]
 [400 450 500 550 600 650]
 [500 550 600 650 700 750]
 [600 650 700 750 800 850]
 [700 750 800 850 900 950]]


Now, we are ready to Go!

We can now define an MLP model for the multivariate input where the vector length is used for the input dimension argument.

In [0]:
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

When we ask the model to make a prediction, the model expects three time steps for two input time series. 

In [17]:
# fit model
model.fit(X, y, epochs=2000, verbose=0)

In [18]:
# demonstrate prediction
x_input = array([[700, 750], [800, 850], [900, 950]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[1855.244]]


**Note:** Given the stochastic nature of this algorithm, your  results may vary.